In [1]:
!pip install --quiet google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: C:\Users\moknotebook\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [ ]:
import json
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import datetime

# Load the credentials.json file
with open('credentials.json') as cred_file:
    creds_data = json.load(cred_file)

# Authenticate and create the service
flow = InstalledAppFlow.from_client_config(creds_data, scopes=['https://www.googleapis.com/auth/calendar'])
creds = flow.run_local_server(port=0)

service = build('calendar', 'v3', credentials=creds)

# Your calendar ID (usually your email)
calendar_id = '10614405@learner.hkuspace.hku.hk'

# Shift data
shifts = [
    {"date": "2024-10-01", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-06", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-07", "start": "15:00", "end": "20:15"},
    {"date": "2024-10-08", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-11", "start": "15:00", "end": "20:15"},
    {"date": "2024-10-12", "start": "12:00", "end": "19:30"},
    {"date": "2024-10-13", "start": "06:30", "end": "15:00"},
    {"date": "2024-10-14", "start": "15:00", "end": "20:15"},
    {"date": "2024-10-15", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-19", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-20", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-21", "start": "06:00", "end": "15:00"},
    {"date": "2024-10-22", "start": "06:00", "end": "15:00"},
    {"date": "2024-10-25", "start": "07:00", "end": "15:00"},
    {"date": "2024-10-26", "start": "15:00", "end": "20:15"},
    {"date": "2024-10-27", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-28", "start": "12:00", "end": "20:15"},
    {"date": "2024-10-29", "start": "12:00", "end": "13:15"}
]

# Function to add or update an event
def add_or_update_event(shift):
    date = shift["date"]
    start_time = datetime.datetime.strptime(date + " " + shift["start"], "%Y-%m-%d %H:%M")
    end_time = datetime.datetime.strptime(date + " " + shift["end"], "%Y-%m-%d %H:%M")
    
    event = {
        'summary': 'Aldi Shift',
        'start': {
            'dateTime': start_time.isoformat(),
            'timeZone': 'Asia/Hong_Kong',
        },
        'end': {
            'dateTime': end_time.isoformat(),
            'timeZone': 'Asia/Hong_Kong',
        },
        'colorId': '7',  # Peacock color
    }

    # Check if there's already an event for this day
    events_result = service.events().list(calendarId=calendar_id, timeMin=start_time.isoformat() + 'Z', timeMax=(end_time + datetime.timedelta(days=1)).isoformat() + 'Z', singleEvents=True).execute()
    events = events_result.get('items', [])

    if events:
        # Update the existing event
        event_id = events[0]['id']
        service.events().update(calendarId=calendar_id, eventId=event_id, body=event).execute()
    else:
        # Create a new event
        service.events().insert(calendarId=calendar_id, body=event).execute()

# Add or update events for each shift
for shift in shifts:
    add_or_update_event(shift)

# Calculate working hours for every fourteen days
def calculate_fortnightly_hours():
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    two_weeks_ago = (datetime.datetime.utcnow() - datetime.timedelta(days=14)).isoformat() + 'Z'

    events_result = service.events().list(calendarId=calendar_id, timeMin=two_weeks_ago, timeMax=now, singleEvents=True).execute()
    events = events_result.get('items', [])

    total_hours = 0
    for event in events:
        start = datetime.datetime.fromisoformat(event['start']['dateTime'][:-1])
        end = datetime.datetime.fromisoformat(event['end']['dateTime'][:-1])
        total_hours += (end - start).total_seconds() / 3600

    return total_hours

# Print working hours for the last fourteen days
print(f"Total working hours for the last 14 days: {calculate_fortnightly_hours()} hours")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=480459223107-p5bn99g8ig5spfs8kshbuafr31gdvqa1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58500%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=UprvBscXvPHOaQKpKZys4EeuhjyMTd&access_type=offline
